# Feature Matching

In [ ]:
import os
import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt

from f3.tasks.matching import StereoEventFFMatcher
from f3.utils import setup_torch, unnormalize_events, BaseExtractor

setup_torch(cudnn_benchmark=False)

# CAR
config = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/config.yml"
ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/last.pth"
# config = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaynightm3ed_small_20ms/models/config.yml"
# ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaynightm3ed_small_20ms/models/last.pth"
# config = "/home/richeek/GitHub/f3/outputs/unetff_fullcardaym3ed_small_20ms/models/config.yml"
# ckpt = "/home/richeek/GitHub/f3/outputs/unetff_fullcardaym3ed_small_20ms/models/last.pth"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_city_hall_data/car_urban_day_city_hall_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_city_hall_data/50khz_car_urban_day_city_hall_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_rittenhouse_data/car_urban_day_rittenhouse_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_rittenhouse_data/50khz_car_urban_day_rittenhouse_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_penno_small_loop_data/car_urban_day_penno_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_penno_small_loop_data/50khz_car_urban_day_penno_small_loop_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_ucity_small_loop_data/car_urban_day_ucity_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_ucity_small_loop_data/50khz_car_urban_day_ucity_small_loop_data.npy"

# NIGHT
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_ucity_small_loop_data/car_urban_night_ucity_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_ucity_small_loop_data/50khz_car_urban_night_ucity_small_loop_data.npy"
h5path = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/car_urban_night_rittenhouse_data.h5"
tspath = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/50khz_car_urban_night_rittenhouse_data.npy"

# SPOT
# config = "/home/richeek/GitHub/f3/outputs/patchff_fullspotindooroutdoorm3ed_small_20ms/models/config.yml"
# ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullspotindooroutdoorm3ed_small_20ms/models/last.pth"

# h5path = "/home/richeek/GitHub/f3/data/spot_indoor_building_loop/spot_indoor_building_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/spot_indoor_building_loop/50khz_spot_indoor_building_loop_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/spot_indoor_obstacles/spot_indoor_obstacles_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/spot_indoor_obstacles/50khz_spot_indoor_obstacles_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/spot_outdoor_night_penno_short_loop/spot_outdoor_night_penno_short_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/spot_outdoor_night_penno_short_loop/50khz_spot_outdoor_night_penno_short_loop_data.npy"

# FALCON
# config = "/home/richeek/GitHub/f3/outputs/patchff_fullfalconindooroutdoorm3ed_small_20ms/models/config.yml"
# ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullfalconindooroutdoorm3ed_small_20ms/models/last.pth"

# h5path = "/home/richeek/GitHub/f3/data/falcon_indoor_flight_1/falcon_indoor_flight_1_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/falcon_indoor_flight_1/50khz_falcon_indoor_flight_1_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/falcon_indoor_flight_2/falcon_indoor_flight_2_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/falcon_indoor_flight_2/50khz_falcon_indoor_flight_2_data.npy"


h5file = h5py.File(h5path, 'r')
tsfile = np.load(tspath, allow_pickle=True)[()]

extractor_left = BaseExtractor(h5path, tspath, w=1280, h=720, time_ctx=20000,
                               time_pred=20000, bucket=1000, max_numevents_ctx=800000,
                               randomize_ctx=False, camera="left")

extractor_right = BaseExtractor(h5path, tspath, w=1280, h=720, time_ctx=20000,
                               time_pred=20000, bucket=1000, max_numevents_ctx=800000,
                               randomize_ctx=False, camera="right")

In [ ]:
img_size        = (1280, 720)
left_calib      = 'prophesee/left/calib'
right_calib     = 'prophesee/right/calib'
Kl              = h5file[left_calib + '/intrinsics'][()]
Kr              = h5file[right_calib + '/intrinsics'][()]
distl           = h5file[left_calib + '/distortion_coeffs'][()]
distr           = h5file[right_calib + '/distortion_coeffs'][()]
l_to_propheseel = h5file[left_calib + '/T_to_prophesee_left'][()]
r_to_propheseel = h5file[right_calib + '/T_to_prophesee_left'][()]
r2l             = np.linalg.inv(r_to_propheseel @ l_to_propheseel)

matcher_config = {
    "minDisparity": -64,
    "numDisparities": 128,
    "blockSize": 9,
    "P1": 8 * 3 * 9**2,
    "P2": 32 * 3 * 9**2,
    "disp12MaxDiff": 0,
    "uniquenessRatio": 5,
    "speckleWindowSize": 200,
    "speckleRange": 2,
    "mode": cv2.StereoSGBM_MODE_HH4,
}

In [ ]:
stereo_evff_matcher = StereoEventFFMatcher(
    eventff_config=config,
    eventff_ckpt=ckpt,
    img_size=img_size,
    Kl=Kl,
    Kr=Kr,
    distl=distl,
    distr=distr,
    r2l=r2l,
    matcher_config=matcher_config,
)

In [ ]:
time = int(10 * 1e6) # in us
ctx_left, totcnt_left = extractor_left.get_ctx_fixedtime(time)
ctx_right, totcnt_right = extractor_right.get_ctx_fixedtime(time)

events_left = unnormalize_events(ctx_left.cpu().numpy(), [1280, 720, 20])
events_right = unnormalize_events(ctx_right.cpu().numpy(), [1280, 720, 20])

event_frame = np.zeros((1280, 720, 3), dtype=np.uint8)
event_frame[events_left[:,0], events_left[:,1], 0] = 255
event_frame[events_right[:,0], events_right[:,1], 1] = 255

plt.imshow(event_frame.transpose(1, 0, 2))
plt.show()

In [ ]:
from tqdm import tqdm
from IPython.display import display, Image

display_handle = display(None, display_id=True)

skip = 10
ovc_images = extractor_left.hdf5_file["ovc/rgb/data"]

foldername = h5path.split("/")[-1].split(".")[0]
folderpath = "/home/richeek/GitHub/f3/outputs/matching/" + foldername
os.makedirs(folderpath, exist_ok=True)

for idx in tqdm(range(10, ovc_images.shape[0], skip)):
    t0 = extractor_left.hdf5_file["ovc/ts"][idx]
    img = ovc_images[idx]
    ctx_left, totcnt_left = extractor_left.get_ctx_fixedtime(t0)
    ctx_right, totcnt_right = extractor_right.get_ctx_fixedtime(t0)

    if totcnt_left < 200000 or totcnt_right < 200000:
        # print("Not enough events")
        continue

    events_left = unnormalize_events(ctx_left.cpu().numpy(), [1280, 720, 20])
    events_right = unnormalize_events(ctx_right.cpu().numpy(), [1280, 720, 20])

    event_frame = np.zeros((1280, 720, 3), dtype=np.uint8)
    event_frame[events_left[:, 0], events_left[:, 1], 0] = 255
    event_frame[events_right[:, 0], events_right[:, 1], 1] = 255

    ldisparity, pca_left_rect, pca_right_rect = stereo_evff_matcher.compute_disparity(
        ctx_left, ctx_right, use_pca=True, return_features=True
    )
    ldisparity_colored = stereo_evff_matcher.process_disparity(
        ldisparity, colormap=cv2.COLORMAP_INFERNO, remove_outliers=True
    )

    # print(f"Disparity stats: min={ldisparity.min()}, max={ldisparity.max()}, mean={ldisparity.mean()}")
    # ldisparity_colored = stereo_evff_matcher.process_disparity(ldisparity, colormap=cv2.COLORMAP_INFERNO)

    cv2.imwrite(f"{folderpath}/image_{idx}.png", img)
    cv2.imwrite(f"{folderpath}/events_lr_{idx}.png", event_frame.transpose(1, 0, 2)[..., ::-1])
    cv2.imwrite(f"{folderpath}/feature_left_{idx}.png", pca_left_rect)
    cv2.imwrite(f"{folderpath}/disparity_{idx}.png", ldisparity_colored)

    # img = cv2.resize(img, (1280, 720))
    # joined = cv2.hconcat([img, ldisparity_colored])
    # _, frame = cv2.imencode('.jpeg', joined)
    # display_handle.update(Image(data=frame.tobytes()))